# Chapter 1: Exploratory Data Analysis of the U.S. Job Market

## Learning Objectives

By the end of this chapter, you will be able to:
- Load and explore the OEWS (Occupational Employment and Wage Statistics) dataset
- Create meaningful visualizations to understand job market trends
- Identify target occupations and states for deeper analysis
- Use data-driven insights to guide subsequent scraping activities

---

## Introduction to the OEWS Dataset

> **Instructor Cue:** Start by opening the BLS website (https://www.bls.gov/oes/) in your browser. Show the attendees where this data comes from and emphasize its credibility as a government source. Mention that this dataset is updated annually and covers over 800 occupations across all states.

The U.S. Bureau of Labor Statistics (BLS) Occupational Employment and Wage Statistics (OEWS) program provides employment and wage estimates for hundreds of occupations. This dataset serves as our foundation for understanding the national job landscape before we dive into targeted data collection.

### Key Dataset Columns

Before we begin our analysis, let's understand the structure of our data:

- `occ_code`: Occupation classification code
- `occ_title`: Occupation title (e.g., "Data Scientists")
- `o_group`: Occupation group level (detailed, minor, major)
- `tot_emp`: Total employment
- `jobs_1000`: Employment per 1,000 jobs
- `a_mean`: Annual mean wage
- `h_mean`: Hourly mean wage
- `state`: State abbreviation

> **Instructor Cue:** Ask the audience: "What questions might we want to ask about this data? What patterns would be interesting to explore?" Write their responses on a whiteboard to refer back to during the analysis.

---

## Setting Up Our Environment

Let's start by importing the necessary libraries and loading our dataset:

In [ ]:
import zipfile
from datetime import timedelta
from pathlib import Path

import matplotlib.pyplot as plt
import pandas as pd
import requests
import requests_cache
import seaborn as sns

# Set up visualization style
sns.set_style("whitegrid")
plt.rcParams["figure.figsize"] = (12, 8)  # Set default matplotlib figure size

# --- Caching Setup ---
# Cache requests to avoid re-downloading the same file from the web
# The cache will be stored in '.requests_cache' and expire after 1 day.
requests_cache.install_cache(".requests_cache", expire_after=timedelta(days=1))

BASE_DIR = Path().cwd()

print(f"{BASE_DIR=}")

> **Instructor Cue:** Emphasize that we're using seaborn with matplotlib's object-oriented API throughout this workshop. This is a best practice for creating publication-quality visualizations. Also highlight that we'll first download the data directly from the BLS website as an example of programmatic data acquisition.

### Downloading Data Directly from the Source

Before we analyze the data, let's practice an important data acquisition skill: downloading data programmatically from its source. The Bureau of Labor Statistics (BLS) provides the OEWS dataset as a downloadable ZIP file.

> **Instructor Cue:** Explain that in real-world scenarios data scientists often need to fetch data from original sources. This ensures we're working with the most current information and builds important data collection skills.

In [ ]:
# Define data source and local storage paths
ZIP_URL = "https://www.bls.gov/oes/special-requests/oesm24ma.zip"
DATA_DIR = BASE_DIR / "data"

# PROTIP: The pathlib.Path object can be used work with URLs to extract components
zip_filename = Path(ZIP_URL).name
zip_path = DATA_DIR / zip_filename

# Make sure the data directory exists
DATA_DIR.mkdir(exist_ok=True)

In [ ]:
print(f"Requesting OEWS dataset from {ZIP_URL}...")

# Set headers to mimic a web browser request
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/122.0.0.0 Safari/537.36",
    "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8",
    "Referer": "https://www.bls.gov/oes/special-requests/",
}

try:
    # Download the file with a streaming approach to handle large files
    response = requests.get(ZIP_URL, headers=headers, stream=True, timeout=30)
    response.raise_for_status()

    # Check if the response was from the cache
    if getattr(response, "from_cache", False):
        print("Data was served from local cache.")
    else:
        print("Data was fetched from the network.")

    with open(zip_path, "wb") as f:
        for chunk in response.iter_content(chunk_size=8192):
            if chunk:
                f.write(chunk)

    print(f"Data saved to {zip_path}")
except requests.HTTPError as e:
    print(f"HTTP error: {e}")
    print(
        f"If you still get a 403 error, you may need to download manually from:\n{ZIP_URL}\nand save it to: {zip_path}"
    )
except Exception as e:
    print(f"Download failed: {e}")

Let's unzip our file and see what we downloaded

In [ ]:
def unzip_file(zip_path, extract_to):
    """Extract all contents of a zip file to the specified directory"""

    with zipfile.ZipFile(zip_path, "r") as zip_ref:
        zip_ref.extractall(extract_to)

    print(f"Successfully unzipped {zip_path}")


# Only unzip if the zip file exists
if zip_path.exists():
    unzip_file(zip_path, DATA_DIR)

    # List extracted files to identify the one we need
    print("\nExtracted files:")
    zip_contents = zipfile.ZipFile(zip_path).namelist()

    for i, file in enumerate(zip_contents[:10]):  # Show first 10 files
        if file.endswith("/"):  # Directory
            print(f"- {file}")
        else:  # File
            print(f"\t- {file}")

    if len(zip_contents) > 10:
        print(f"...and {len(zip_contents) - 10} more files")
else:
    print("Zip file not available. Skipping extraction step.")

### Processing the BLS Excel Data

Looking at the extracted files, we can see that the BLS OEWS data comes as multiple Excel files within the ZIP archive. For our analysis, we'll use the `MSA_M2024_dl.xlsx` file which contains metropolitan statistical area data.

> **Instructor Cue:** Explain that Excel files often contain multiple sheets and raw data that needs cleaning. Walk through how we examine the file structure before deciding on our processing approach. Highlight that this transformation from raw data to analysis-ready format is a key data science skill.

In [ ]:
# Let's process the Excel file into a dataframe

excel_path = DATA_DIR / "oesm24ma" / "MSA_M2024_dl.xlsx"

if excel_path.exists():
    # Load the Excel file - using sheet_name=None to get all sheets as a dictionary
    excel_data = pd.read_excel(excel_path, sheet_name=None)

    # Print available sheets to help us understand the file structure
    print(f"Available sheets in the Excel file: {list(excel_data.keys())}")

    # Let's use the main sheet (assuming it's the first one)
    main_sheet = list(excel_data.keys())[0]
    oews_df = excel_data[main_sheet]

    # Display basic information about the dataset
    print(f"\nDataset shape: {oews_df.shape}")

    # Preview the first few rows
    display(oews_df.head())

# Rename our dataframe to df for consistency with the rest of the notebook
df = oews_df

In [ ]:
# Let's examine our dataframe more thoroughly
print("=== Data Information ===")
df.info()

In [ ]:
# Let's get summary statistics for numeric columns
print("=== Summary Statistics ===")
display(df.describe())

## Data Quality Check

> **Instructor Cue:** Walk through the output together. Point out any surprising values or data types. Ask: "What do you notice about the data? Any immediate red flags or interesting patterns?"

In [ ]:
import pprint as pp

# Check for missing values
missing_data = df.isna().sum()
display(
    missing_data[missing_data > 0]
    .to_frame("n_missing_values")
    .style.set_caption("Missing Values by Column")
)

# Check unique values in key categorical columns
print(f"Number of states: {df['PRIM_STATE'].nunique()}")
print(f"Sample states: {df['PRIM_STATE'].unique()[:10]}")
print("\nColumn names for reference:")
pp.pprint(list(df.columns), compact=True, width=120)

## EDA Visualizations

Before we create visualizations, let's clean our data to focus only on complete records and detailed occupations:

In [ ]:
# Convert relevant columns to numeric (handling any non-numeric values)
cols_to_convert = [
    "H_MEAN",
    "A_MEAN",
    "TOT_EMP",
    "JOBS_1000",
    "H_PCT25",
    "H_MEDIAN",
    "H_PCT75",
    "A_PCT25",
    "A_MEDIAN",
    "A_PCT75",
]

for col in cols_to_convert:
    if col in df.columns:
        df[col] = pd.to_numeric(df[col], errors="coerce")
    else:
        print(f"Warning: Column '{col}' not found in dataframe")

# Clean the data for our analysis
# Remove rows with missing wage data and focus on detailed occupations
df_clean = df[
    (df["O_GROUP"] == "detailed")
    & (df["H_MEAN"].notna())
    & (df["A_MEAN"].notna())
    & (df["TOT_EMP"].notna())
    & (df["JOBS_1000"].notna())
].copy()

print(f"Cleaned dataset shape: {df_clean.shape}")
print(f"Removed {df.shape[0] - df_clean.shape[0]} rows with missing data")

# Verify the data types after cleaning
display(
    df_clean[cols_to_convert]
    .dtypes.to_frame("dtypes")
    .style.set_caption("Data Types After Cleaning")
)

### Visualization 1: Top-Paying Occupations by State

Let's compare the highest-paying occupations between Oregon and Louisiana to understand regional differences:

> **Instructor Cue:** Ask the class: "Why might we choose Oregon and Louisiana for comparison? What economic differences might we expect?" Encourage discussion about tech hubs vs. traditional industries.

In [ ]:
def create_wage_comparison_chart(df, states, top_n=20):
    """
    Create a bar chart comparing top-paying occupations between states.

    Args:
        df: Cleaned OEWS dataframe
        states: List of state abbreviations to compare
        top_n: Number of top occupations to display
    """
    # Filter for target states
    state_data = df[df["PRIM_STATE"].isin(states)].copy()

    # Get top paying occupations for each state
    top_occupations = []
    for state in states:
        state_subset = state_data[state_data["PRIM_STATE"] == state]
        top_state = state_subset.nlargest(top_n, "H_MEAN")
        top_state["rank"] = range(1, len(top_state) + 1)
        top_occupations.append(top_state)

    combined_data = pd.concat(top_occupations)

    # Create the visualization
    fig, ax = plt.subplots(figsize=(14, 10))

    # Create bar plot
    sns.barplot(
        data=combined_data,
        y="OCC_TITLE",
        x="H_MEAN",
        hue="PRIM_STATE",
        ax=ax,
        palette="viridis",
    )

    ax.set_title(
        f"Top {top_n} Highest-Paying Occupations: {' vs '.join(states)}",
        fontsize=16,
        fontweight="bold",
    )
    ax.set_xlabel("Hourly Mean Wage ($)", fontsize=12)
    ax.set_ylabel("Occupation Title", fontsize=12)

    # Improve readability
    ax.tick_params(axis="y", labelsize=10)
    plt.tight_layout()

    return fig, combined_data


# Execute the comparison
states_to_compare = ["OR", "LA"]
fig, wage_data = create_wage_comparison_chart(df_clean, states_to_compare)
plt.show()

# Display top 5 for each state
for state in states_to_compare:
    state_top = wage_data[wage_data["PRIM_STATE"] == state].head()
    print(f"\nTop 5 occupations in {state}:")
    print(state_top[["OCC_TITLE", "H_MEAN", "TOT_EMP"]].to_string(index=False))

### Visualization 2: Employment vs. Salary Relationship

Now let's examine how employment levels relate to salary across different states:

> **Instructor Cue:** This visualization will help us understand the relationship between job availability and compensation levels.

In [ ]:
import matplotlib.ticker as mticker


def create_employment_salary_scatter(df, sample_states=None, include_other_states=False):
    """
    Create a scatter plot showing the relationship between employment
    density and annual salary by state.

    Args:
        df: Cleaned OEWS dataframe
        sample_states: List of states to highlight (optional)
    """
    # Filter for states with sufficient data
    state_counts = df["PRIM_STATE"].value_counts()
    states_with_data = state_counts[state_counts >= 50].index

    plot_data = df[df["PRIM_STATE"].isin(states_with_data)].copy()

    # Create the scatter plot
    fig, ax = plt.subplots()

    if sample_states:
        # Highlight specific states
        for state in sample_states:
            state_data = plot_data[plot_data["PRIM_STATE"] == state]
            ax.scatter(
                state_data["JOBS_1000"],
                state_data["A_MEAN"],
                label=state,
                alpha=0.7,
                s=60,
            )

        if include_other_states:
            # Plot other states in grey
            other_states = plot_data[~plot_data["PRIM_STATE"].isin(sample_states)]
            ax.scatter(
                other_states["JOBS_1000"],
                other_states["A_MEAN"],
                color="lightgrey",
                alpha=0.3,
                s=20,
                label="Other States",
            )
    else:
        # Color by state (too many for legend)
        sns.scatterplot(
            data=plot_data,
            x="JOBS_1000",
            y="A_MEAN",
            hue="PRIM_STATE",
            ax=ax,
            legend=False,
            alpha=0.6,
        )

    ax.set_title("Employment Density vs. Annual Salary by State", fontsize=16, fontweight="bold")
    ax.set_xlabel("Employment per 1,000 Jobs", fontsize=12)
    ax.set_ylabel("Annual Mean Wage ($)", fontsize=12)
    ax.yaxis.set_major_formatter(mticker.FuncFormatter(lambda x, _: f"{int(x):,}"))

    if sample_states:
        ax.legend()

    # # Fit a trend line using numpy.polynomial.Polynomial
    # # Remove NaNs for fitting
    # x_fit = plot_data["JOBS_1000"].dropna()
    # y_fit = plot_data["A_MEAN"].dropna()

    # # Fit a 1st degree polynomial (linear trend)
    # coefs = np.polynomial.Polynomial.fit(x_fit, y_fit, 1).convert().coef
    # p = np.polynomial.Polynomial(coefs)
    # ax.plot(x_fit, p(x_fit), "r--", alpha=0.8, linewidth=2, label="Trend Line")

    plt.tight_layout()
    return fig, plot_data


# Create the scatter plot highlighting our target states
fig, scatter_data = create_employment_salary_scatter(df_clean, ["OR", "LA"])
plt.show()

# Analyze the correlation
correlation = scatter_data["JOBS_1000"].corr(scatter_data["A_MEAN"])
print(f"\nCorrelation between employment density and salary: {correlation:.3f}")

### Visualization 3: State Employment Heatmap

Finally, let's create a heatmap to visualize which occupations are most prevalent in different states:

> **Instructor Cue:** This visualization will help us identify patterns of occupational concentration across states.

In [ ]:
def create_occupation_heatmap(df, top_occupations=15, top_states=10):
    """
    Create a heatmap showing total employment by occupation and state.

    Args:
        df: Cleaned OEWS dataframe
        top_occupations: Number of top occupations to include
        top_states: Number of top states to include
    """
    # Find the most prevalent occupations nationally
    national_employment = df.groupby("OCC_TITLE")["TOT_EMP"].sum().sort_values(ascending=False)
    top_occ_list = national_employment.head(top_occupations).index

    # Find states with highest total employment
    state_employment = df.groupby("PRIM_STATE")["TOT_EMP"].sum().sort_values(ascending=False)
    top_state_list = state_employment.head(top_states).index

    # Filter data
    heatmap_data = df[
        (df["OCC_TITLE"].isin(top_occ_list)) & (df["PRIM_STATE"].isin(top_state_list))
    ].copy()

    # Create pivot table
    pivot_table = heatmap_data.pivot_table(
        index="OCC_TITLE", columns="PRIM_STATE", values="TOT_EMP", fill_value=0
    )

    # Create the heatmap
    fig, ax = plt.subplots(figsize=(12, 10))

    sns.heatmap(
        pivot_table,
        annot=True,
        fmt=".0f",
        cmap="YlOrRd",
        ax=ax,
        cbar_kws={"label": "Total Employment"},
    )

    ax.set_title(
        f"Employment Heatmap: Top {top_occupations} Occupations in Top {top_states} States",
        fontsize=14,
        fontweight="bold",
    )
    ax.set_xlabel("State", fontsize=12)
    ax.set_ylabel("Occupation Title", fontsize=12)

    # Improve label readability
    plt.xticks(rotation=45)
    plt.yticks(rotation=0)
    plt.tight_layout()

    return fig, pivot_table


# Create the heatmap
fig, employment_heatmap = create_occupation_heatmap(df_clean, top_occupations=20)
plt.show()

# Show some interesting insights
if "Data Scientists" in employment_heatmap.index:
    print("\nStates with highest employment in Data Science roles:")

    data_sci_employment = employment_heatmap.loc["Data Scientists"].sort_values(ascending=False)
    print(data_sci_employment.head().to_string())
else:
    print("Data Scientists not in top occupations nationally")

In [ ]:
def identify_target_occupations(df, criteria="high_wage_tech"):
    """
    Identify target occupations based on specified criteria.

    For each state, finds the top 5 unique occupations by annual wage,
    regardless of the specific area within the state. When an occupation
    appears in multiple areas, only the highest-paying instance is included.

    Args:
        df: Cleaned OEWS dataframe
        criteria: Selection criteria ('high_wage_tech', 'high_growth', 'regional_strong')
    """
    # Columns to keep in the final output
    target_columns = [
        "OCC_TITLE",
        "AREA_TITLE",
        "PRIM_STATE",
        "A_MEAN",
        "H_MEAN",
        "TOT_EMP",
        "JOBS_1000",
        "H_PCT25",
        "H_MEDIAN",
        "H_PCT75",
        "A_PCT25",
        "A_MEDIAN",
        "A_PCT75",
    ]

    # Initialize empty list to hold selected rows
    selected_rows = []

    if criteria == "high_wage_tech":
        # Focus on high-paying tech roles in specific states
        tech_keywords = "|".join(["Software", "Data", "Computer", "Information", "Web"])
        tech_occupations = df[
            df["OCC_TITLE"].str.contains(tech_keywords, regex=True, case=False, na=False)
        ]

        # Get top-paying tech roles in OR and CA
        for state in ["OR", "CA"]:
            state_tech = tech_occupations[tech_occupations["PRIM_STATE"] == state]

            if not state_tech.empty:
                # Group by OCC_TITLE and find the highest paying instance of each occupation
                # This ensures we get unique occupations regardless of area
                top_occs = (
                    state_tech.groupby("OCC_TITLE")
                    .apply(lambda x: x.nlargest(1, "A_MEAN"))
                    .reset_index(drop=True)
                )
                # Now get the top 5 unique occupations by annual wage
                top_roles = top_occs.nlargest(5, "A_MEAN")
                selected_rows.append(top_roles)

                print(f"Selected {len(top_roles)} unique top-paying occupations from {state}")
            else:
                print(f"Warning: No tech occupations found for state {state}")

    # Combine all selected rows into a single DataFrame
    if selected_rows:
        target_df = pd.concat(selected_rows)

        # Add a rationale column
        target_df["RATIONALE"] = target_df.apply(
            lambda row: f"High-paying tech role in {row['PRIM_STATE']}", axis=1
        )

        # Use only columns that are actually in our data
        return target_df[target_columns + ["RATIONALE"]]
    else:
        # Create empty dataframe with only available columns
        return pd.DataFrame(columns=target_columns + ["RATIONALE"])


target_df = identify_target_occupations(df_clean)


# Format numbers with thousand separator for display
def format_thousands(val):
    if isinstance(val, (int, float)):
        return f"{val:,.0f}"
    return val


# Create a dictionary for formatting numeric columns based on available columns
format_dict = {}
# Basic columns that should always be there
format_dict["A_MEAN"] = format_thousands
format_dict["H_MEAN"] = lambda x: f"${x:.2f}"
format_dict["TOT_EMP"] = format_thousands
format_dict["JOBS_1000"] = lambda x: f"{x:.1f}" if pd.notnull(x) else ""

# Add formatting for columns that may or may not be present
hourly_cols = ["H_PCT25", "H_MEDIAN", "H_PCT75"]
for col in hourly_cols:
    format_dict[col] = lambda x: f"${x:.2f}" if pd.notnull(x) else ""

annual_cols = ["A_PCT25", "A_MEDIAN", "A_PCT75"]
for col in annual_cols:
    format_dict[col] = format_thousands

display(target_df.style.format(format_dict).set_caption("Target Occupations for Web Scraping"))

> **Instructor Cue:** Explain that this data-driven approach to selecting scraping targets is much more effective than random selection. Ask the class: "How does this analysis inform our scraping strategy? What additional data might we want to collect for these specific roles?"

---

## Key Insights and Next Steps

### Summary of Findings

Let's summarize what we've learned from our exploratory analysis:

In [ ]:
# Generate summary statistics
print("=== EXPLORATORY DATA ANALYSIS SUMMARY ===")
print(f"Total occupations analyzed: {df_clean['OCC_TITLE'].nunique()}")
print(f"States covered: {df_clean['PRIM_STATE'].nunique()}")
print(f"Average annual wage across all occupations: ${df_clean['A_MEAN'].mean():,.0f}")
print(f"Highest paying occupation: {df_clean.loc[df_clean['A_MEAN'].idxmax(), 'OCC_TITLE']}")
print(
    f"Most common occupation by employment: {df_clean.groupby('OCC_TITLE')['TOT_EMP'].sum().idxmax()}"
)

# Save our target occupations for the next module
# Create the data directory if it doesn't exist
DATA_DIR.mkdir(exist_ok=True)

# Print information about the data we're saving
print(f"\nTarget dataframe shape: {target_df.shape}")
columns_to_save = target_df.columns

print("\nSaving the following columns to CSV:")
pp.pprint(list(columns_to_save), compact=True, width=80)

# Ensure we handle the case where the dataframe might be empty
if not target_df.empty:
    target_df.to_csv(DATA_DIR / "bls_jobs_metro_area_2024.csv", index=False)

    print("Target occupations saved to 'data/bls_jobs_metro_area_2024.csv'")
else:
    print("Warning: No target occupations identified. CSV file not created.")

print("Ready for targeted web scraping in Chapter 2!")

### Transition to Web Scraping

> **Instructor Cue:** This is a perfect transition moment. Emphasize that we now have a clear, data-driven rationale for our scraping targets. Ask: "What specific information might we want to collect about these roles that isn't in the BLS data?"

Our exploratory analysis has revealed:

1. **Regional Specialization**: Different states show clear patterns in high-paying occupations
2. **Employment vs. Wage Dynamics**: Understanding the relationship between job availability and compensation
3. **Target Identification**: Data-driven selection of specific occupation-state combinations

In the next chapter, we'll use web scraping to collect real-time job posting data for our identified targets, adding depth to our government dataset with current market information including specific requirements, company details, and salary ranges.

---

## Quick Practice Exercise

> **Instructor Cue:** This is a 5-minute exercise to reinforce the analysis concepts. Have students work in pairs and share their findings quickly.

**Exercise: Identify Your Own Scraping Target**

Using the analysis functions we've built, pick two states of your choice and identify one high-paying occupation that would be interesting to scrape in Chapter 2.

In [ ]:
# Quick exercise: Choose your states and find an interesting target
my_states = ["WA", "FL", "OH"]  # Replace with your choice

# Use our existing functions to analyze these states
fig, my_wage_data = create_wage_comparison_chart(df_clean, my_states, top_n=10)
plt.show()

# Pick one occupation that interests you for scraping
print("\nMy chosen target for web scraping:")
print("Occupation: [Your choice here]")
print("State: [Your choice here]")
print("Rationale: [Why this is interesting]")

> **Instructor Cue:** Have 2-3 students quickly share their chosen targets. This will give us variety in the scraping exercise in Chapter 2.

---

*End of Chapter 1*

> **Instructor Cue:** Before moving to Chapter 2, do a quick check-in: "Any questions about the analysis we just performed? Are you feeling confident about the patterns we identified?" This is also a good time for a 10-minute break.